<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/equal_realize/Continue_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
!pip install keras_bert

  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=4f5a7e1c9ce3b3816ec8c60935d8fd614cc262f5876e36d6847151adc425cb8e
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for keras-transformer: filename=keras_transformer-0.32.0-cp36-none-any.whl size=13266 sha256=2d534f7c8cf74e2c4157994ef75c6d3032f133f8b28ba55c4a44986d6cafc7e2
  Stored in directory: /root/.cache/pip/wheels/62/f0/ce/82fa5d024d5ef8e263f26a50dcee23820efe245680ce9c922a
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=1787c123607bab7a30c987c72c6d0201e48a62949ec562619b0982a090045267
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=e73ddcf89550b44685d37bf9c337d65dda418ccd7dcd071f852de442b1989096
  Stor

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-ic5oj11p
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-ic5oj11p
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=b1134ebabb21e47d4c407475da0728b218a82592f6b82473e746d1275f8cdb3f
  Stored in directory: /tmp/pip-ephem-wheel-cache-a_jv35bp/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import os
os.chdir("/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/")
lines = []
with open("crf.py", "r") as fd:
  lines = fd.readlines()
lines[515] = "            mask2 = K.cast(K.concatenate([mask, K.cast(K.zeros_like(mask[:, :1]), mask.dtype)], axis=1),\n"
with open("crf.py", "w") as fd:
  fd.writelines(lines)

os.chdir("/content")

In [0]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

Cloning into 'bachelor_design'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 328 (delta 50), reused 62 (delta 34), pack-reused 243
Receiving objects: 100% (328/328), 5.98 MiB | 19.56 MiB/s, done.
Resolving deltas: 100% (181/181), done.
/content/bachelor_design


In [0]:
!git checkout equal_realize
!git pull

Branch 'equal_realize' set up to track remote branch 'equal_realize' from 'origin'.
Switched to a new branch 'equal_realize'
Already up to date.


# Mount GDriver
Perpare Colab environment.

In [0]:
from driver_amount import addh

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
[Colab] Using address head: /gdrive/My Drive


# Load Params

In [0]:
import json
import config

In [0]:
tag_vocab = None
with open(addh + config.TAG_VOCAB_PATH, "r") as fd:
  tag_vocab = json.load(fd)

# Prepare Input

In [0]:
from cut_and_tag import cut_and_tag
import preprocess

In [0]:
cut_seqs, char_seqs, tag_seqs = cut_and_tag(
    addh + config.DATA_PATH,
    addh + config.STOPWORDS_PATH
)
char_seqs, tag_seqs = preprocess.shuffle_twin(
    char_seqs,
    tag_seqs
)
token_id_seqs, segment_seqs, tag_id_seqs, tag_vocab = preprocess.preprocess(
    char_seqs, 
    tag_seqs,
    addh + config.BERT_VOCAB_PATH,
    SEQ_LEN=config.SEQ_LEN,
    tag_vocab=tag_vocab
    )

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.792 seconds.
Prefix dict has been built successfully.
Using TensorFlow backend.


In [0]:
# Split train and text
train_num = int(len(token_id_seqs) * 0.9)

test_token_id_seqs = token_id_seqs[train_num:]
test_segment_seqs = segment_seqs[train_num:]
test_tag_id_seqs = tag_id_seqs[train_num:]

train_token_id_seqs = token_id_seqs[0:train_num]
train_segment_seqs = segment_seqs[0:train_num]
train_tag_id_seqs = tag_id_seqs[0:train_num]

In [0]:
tag_vocab

{'': 0,
 'B-EI': 2,
 'B-EIF': 10,
 'B-EO': 6,
 'B-EQ': 8,
 'B-ILF': 4,
 'I-EI': 3,
 'I-EIF': 11,
 'I-EO': 7,
 'I-EQ': 9,
 'I-ILF': 5,
 'O': 1}

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [0]:
bert_model, bert_model_config = keras_bert.build_model_from_config(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True
)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [0]:
from keras_contrib.layers import CRF

In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

crf_model = CRF(len(tag_vocab), sparse_target=True)

output = crf_model(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [0]:
model.load_weights(addh + config.MODEL_PATH)

# Load train record

In [0]:
import json
with open(addh + config.TRAIN_REC_PATH, "r") as fd:
    train_rec = json.load(fd)
last_epoch = 0
if len(train_rec) > 1:
    last_epoch = train_rec[-1]["epoch"]

# Fine-tuning

In [0]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=crf_model.loss_function,
    metrics=[crf_model.accuracy]
)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [0]:
import epoch_checkpoint
epoch_callback = epoch_checkpoint.EpochCheckpoint(
    addh + config.MODEL_PATH,
    addh + config.TRAIN_REC_PATH,
    1,
    [test_token_id_seqs, test_segment_seqs],
    test_tag_id_seqs,
    tag_vocab
    )

In [0]:
batch_size = 16
completed = False
import tensorflow as tf

In [29]:
while not completed:
  try:
    model.fit(
        x=[train_token_id_seqs, train_segment_seqs],
        y=[train_tag_id_seqs], 
        batch_size=8,
        initial_epoch=last_epoch+1,
        epochs=train_rec[0]["train_params"]["epochs"],
        verbose=1,
        validation_data=[[test_token_id_seqs, test_segment_seqs], [test_tag_id_seqs]],
        callbacks=[epoch_callback]
    )
    completed = True
  except tf.errors.ResourceExhaustedError:
    batch_size /= 2
    print("Batch Size too large, turn to " + str(batch_size))

Train on 7005 samples, validate on 779 samples
Epoch 7/15
Batch Size too large, turn to 8
Train on 7005 samples, validate on 779 samples
Epoch 7/15
Batch Size too large, turn to 4
Train on 7005 samples, validate on 779 samples
Epoch 7/15
Batch Size too large, turn to 2
Train on 7005 samples, validate on 779 samples
Epoch 7/15
7005/7005 [==============================] - 5147s 735ms/step - loss: 55.7178 - crf_viterbi_accuracy: 0.9053 - val_loss: 55.4849 - val_crf_viterbi_accuracy: 0.9115
Saving checkpint...
Predicing for matrics calculating...
779/779 [==============================] - 202s 260ms/step
Updating train record...
Epoch 8/15
 942/7005 [===>..........................] - ETA: 1:11:47 - loss: 55.2049 - crf_viterbi_accuracy: 0.9289

KeyboardInterrupt: ignored

In [0]:
model.save(addh + config.MODEL_PATH)